# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [31]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [32]:
vacation_df = pd.read_csv('Cities_Data.csv')
vacation_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Port Elizabeth,-33.9180,25.5701,23.00,83,75,8.23,ZA,1612366424
1,Bāglung,28.2719,83.5898,9.86,49,16,2.33,NP,1612366323
2,Tasiilaq,65.6145,-37.6368,-2.36,97,100,9.48,GL,1612366424
3,Vaini,-21.2000,-175.2000,25.00,94,75,2.74,TO,1612366425
4,Pevek,69.7008,170.3133,-22.17,76,58,3.61,RU,1612366425


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [33]:
gmaps.configure(api_key=g_key)

In [34]:
locations = vacation_df[["Lat", "Lng"]].astype(float)
humidity = vacation_df["Humidity"]

In [37]:
vacation_df.info()

fig = gmaps.figure(center=(50.0, -5.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=20,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 570 entries, 0 to 569
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        570 non-null    object 
 1   Lat         570 non-null    float64
 2   Lng         570 non-null    float64
 3   Max Temp    570 non-null    float64
 4   Humidity    570 non-null    int64  
 5   Cloudiness  570 non-null    int64  
 6   Wind Speed  570 non-null    float64
 7   Country     565 non-null    object 
 8   Date        570 non-null    int64  
dtypes: float64(4), int64(3), object(2)
memory usage: 40.2+ KB


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [38]:
vacation_destinations =  vacation_df.loc[(vacation_df['Cloudiness'] == 0) & (vacation_df['Max Temp'] < 27 ) & (vacation_df['Max Temp'] > 21 ) & (vacation_df['Wind Speed'] < 5)]

In [39]:
vacation_destinations.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 12 entries, 23 to 505
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        12 non-null     object 
 1   Lat         12 non-null     float64
 2   Lng         12 non-null     float64
 3   Max Temp    12 non-null     float64
 4   Humidity    12 non-null     int64  
 5   Cloudiness  12 non-null     int64  
 6   Wind Speed  12 non-null     float64
 7   Country     12 non-null     object 
 8   Date        12 non-null     int64  
dtypes: float64(4), int64(3), object(2)
memory usage: 960.0+ bytes


In [40]:
vacation_destinations

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
23,Bredasdorp,-34.5322,20.0403,23.00,53,0,4.12,ZA,1612366427
51,Kruisfontein,-34.0033,24.7314,26.67,66,0,2.68,ZA,1612366430
74,Hermanus,-34.4187,19.2345,24.44,56,0,2.24,ZA,1612366434
141,Rāzole,16.4833,81.8333,24.00,73,0,1.03,IN,1612366442
155,Port Alfred,-33.5906,26.8910,25.00,84,0,0.89,ZA,1612366444
281,Pisco,-13.7000,-76.2167,23.00,78,0,4.63,PE,1612366457
328,Salalah,17.0151,54.0924,24.00,50,0,1.54,OM,1612366462
401,Manzanillo,20.3433,-77.1167,22.92,49,0,3.67,CU,1612366469
423,Adelaide,-34.9333,138.6000,22.78,56,0,4.12,AU,1612366208
425,Dubai,25.2582,55.3047,22.00,68,0,4.12,AE,1612366318


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [42]:
params = { "radius": 50000,
            "types" : "lodging",
            "key": g_key,
            "keyword": "hotels"
         }

for index, row in vacation_destinations.iterrows():
    lat = row['Lat']
    lng = row['Lng']

    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    Hotel = requests.get(base_url, params=params).json()
    try:
        vacation_destinations.loc[index,"Hotel Name"] = Hotel["results"][0]["name"]
    except (KeyError, IndexError):
        print("Hotel Not Found!")

In [43]:
vacation_destinations.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
23,Bredasdorp,-34.5322,20.0403,23.00,53,0,4.12,ZA,1612366427,Oceans Guest House & Luxurious Apartments
51,Kruisfontein,-34.0033,24.7314,26.67,66,0,2.68,ZA,1612366430,The Sands @ St. Francis Bay
74,Hermanus,-34.4187,19.2345,24.44,56,0,2.24,ZA,1612366434,Kennedys Beach Villa
141,Rāzole,16.4833,81.8333,24.00,73,0,1.03,IN,1612366442,RVR Sarovar Portico Dindi
155,Port Alfred,-33.5906,26.8910,25.00,84,0,0.89,ZA,1612366444,Pumba Private Game Reserve


In [44]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in vacation_destinations.iterrows()]
locations = vacation_destinations[["Lat", "Lng"]]

In [26]:
# Add marker layer ontop of heat map
hotel_marker = gmaps.marker_layer(
    locations, info_box_content = hotel_info, hover_text=hotel_info
)


fig = gmaps.figure(center=(50.0, -5.0), zoom_level=2)

fig.add_layer(heat_layer)
fig.add_layer(hotel_marker)
fig

Figure(layout=FigureLayout(height='420px'))